In [1]:
import cv2
import numpy as np

In [2]:
def getPicture(filename):
    img = cv2.imread(filename) 
    lis = img.shape     
    piclist = []
    row = lis[0]
    col = lis[1]
    for i in range(row):
        for j in range(col):
            pix_b = img[i][j][0]
            pix_g = img[i][j][1]
            pix_r = img[i][j][2]
            piclist += [[i, j, pix_r, pix_g, pix_b]]
    return piclist


The function `getPicture(filename)` is designed to convert the three channels of an image, which represent the red, green, and blue (RGB) color components, into a single channel.<BR> Initially, the dimensions of the image are three-dimensional (row,column,3), with the third dimension representing the RGB channels. In OpenCV (cv2), these channels are represented as BGR (blue, green, red). The piclist variable is used to store the information of all the pixels in the image. Each pixel is represented by its row and column coordinates, followed by the intensity values for red, green, and blue. For example, for a 1024x1024 RGB image, each pixel would be represented as [row, column, red, green, blue].

In [3]:
def putDataInPixel(index, sixBinary, pixels):
    pixels[index][2] &= 252 #clearing blue component lsb
    pixels[index][3] &= 252 #clearing green component lsb
    pixels[index][4] &= 252 #clearing red component lsb
    pixels[index][2] |= int(sixBinary[0:2], 2) #storing msg in cleared bit
    pixels[index][3] |= int(sixBinary[2:4], 2)
    pixels[index][4] |= int(sixBinary[4:6], 2)


## putDataInPixel
The putDataInPixel function is responsible for encoding data into the least significant bits (LSB) of the pixel values
```python
    pixels[index][2] &= 252
    pixels[index][3] &= 252
    pixels[index][4] &= 252
```
These lines clear the least significant 2 bits (LSB) of the blue, green, and red components of the pixel at the specified index.<br>This is done by performing a bitwise AND operation with the binary value 11111100, which sets the last two bits to 0.<br>
```python
    pixels[index][2] |= int(sixBinary[0:2], 2)
```
above line sets the last two bits of the blue component of the pixel at the specified index to the first two bits of the sixBinary string. It does this by performing a bitwise OR operation with the binary value of the first two characters of the sixBinary string.




In [4]:
def stg(message,pixels):
    messageLength = len(message)
    messageLengthBin = bin(messageLength)[2:]
    messageLengthBin = (24 - len(messageLengthBin)) * "0" + messageLengthBin
    pixelIndex = 0
    
    for i in range(0, 24, 6):
        putDataInPixel(pixelIndex, messageLengthBin[i: i + 6],pixels)
        #print(pixels[pixelIndex])
        pixelIndex += 1
 
    messageInBin = ""
    for char in message:
        binaryStr = bin(ord(char))[2:]
        binaryStr = (8 - len(binaryStr)) * "0" + binaryStr # ascii only 127 i.e 0111111
        messageInBin += binaryStr
        messageInBin = (6 - (len(messageInBin) % 6)) * "0" + messageInBin # to make multiple of 6

    pixelIndex += 1
    for i in range(0, len(messageInBin), 6):
        putDataInPixel(pixelIndex, messageInBin[i: i + 6], pixels)
        pixelIndex += 1

## stg
above function first reads the length of message and stores them in LSB of Red, blue and green component [Message length is stored because it can be useful if we wished to decrypt the information]. After that it converts the text informatino into ascii and stores them in LSB. Befor storing ASCII it adds one extra bit infront of obtained ASCII,as ASCII is representted by 7 bits. Also 4 zeros are added infront of that 8 bit ASCII to make if multiple of 6 because we use 6 bit[red 2 bit, green 2 bit, blue 2 bit] to store information.


In [10]:
def makePicture(pic):
    img1 = []
    img2 = []
    row = pic[-1][0] + 1
    col = pic[-1][1] + 1
    for i in range(row):
        for j in range(col):
            index = i * col + j
            img1 += [[np.uint8(pic[index][4]), np.uint8(pic[index][3]), np.uint8(pic[index][2])]]
        img2 += [img1]
        img1 = []
    #print(np.array(img2).shape)
    return np.array(img2)

## makepicture
This function does opposite of getpicture function.
This converts that [rows, column, R G, B] into its orginal 3 dimensional form

In [6]:
def main():
    pictureName='pic.jpg'     #enter name of your image stored in same directory
    messageText="hello"    #enter message to hide   
    pixels=[]
    pixels += getPicture(pictureName) #get all 3 channels in one list
    stg(messageText, pixels)         #stores message length and message in LSBs
    encryptedPicture = makePicture(pixels)     #changes back to 3 channel form
    name='desiredName.jpg'     #enter name for output
    cv2.imwrite(name, encryptedPicture)       # writes encrypted image
      

In [9]:
main()


(1024, 1024, 3)


## `Compiled by` _Sachin Koirala_

